# MNIST from Scratch

Can I train a model to recognize handwritten digits using numpy?

1. Load the MNIST dataset from the web and store as NumPy arrays
2. Train a simple model to solve MNIST using PyTorch
3. Do the same with NumPy by implementing various ML algorithms

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import trange
from tinygrad.tensor import Tensor
from tinygrad.nn.optim import Adam, get_parameters
from datasets import fetch_mnist
from extra.training import train, evaluate
np.random.seed(1337)
Tensor.manual_seed(1337)

triton backend not available No module named 'pycuda'


Load mnist dataset

In [2]:
X_train, Y_train, X_test, Y_test = fetch_mnist()
X_train /= 255.0
X_test /= 255.0

Solve with PyTorch

In [3]:
class NeuralNet:
    def __init__(self):
        super().__init__()
        self.layer1 = Tensor.scaled_uniform(784, 128)
        self.layer2 = Tensor.scaled_uniform(128, 10)
    
    def forward(self, x: Tensor) -> Tensor:
        x = x.reshape(-1, 28 * 28)
        x = x.dot(self.layer1).relu()
        x = x.dot(self.layer2).logsoftmax()
        return x

In [4]:
iterations = 1000
batch_size = 256
learning_rate = 0.001
model = NeuralNet()
optimizer = Adam(get_parameters(model), lr=learning_rate)

In [5]:
# training loop
train(model, X_train, Y_train, optimizer, iterations)

loss 0.16 accuracy 0.95: 100%|██████████| 1000/1000 [00:29<00:00, 33.90it/s]


In [6]:
# test accuracy
evaluate(model, X_test, Y_test);

100%|██████████| 79/79 [00:00<00:00, 157.72it/s]

test set accuracy is 0.948000


Sanity check with NumPy

In [7]:
w1 = model.layer1.detach().numpy().astype(np.float64)
w2 = model.layer2.detach().numpy().astype(np.float64)

In [8]:
def forward(x: np.ndarray) -> np.ndarray:
    x = x @ w1
    x = np.maximum(x, 0)
    x = x @ w2
    return x

In [9]:
pred = forward(X_test).argmax(axis=1)
accuracy = (pred == Y_test).mean()
accuracy

0.948

Now solve with NumPy

In [10]:
def layer_init(m: int, h: int) -> np.ndarray:
    weights = np.random.uniform(-1., 1., size=(m, h)) / np.sqrt(m * h)
    return weights

In [11]:
w1 = layer_init(784, 128)
w2 = layer_init(128, 10)

In [12]:
def saved_forward(x0: np.ndarray) -> tuple[np.ndarray]:
    x1 = x0 @ w1  # batch_size * 128
    x2 = np.maximum(x1, 0)  # batch_size * 128, relu
    x3 = x2 @ w2  # batch_size * 10
    x3 = np.exp(x3) / np.sum(np.exp(x3), axis=1, keepdims=True)  # softmax
    return x3, x2, x1, x0

In [13]:
def cross_entropy_loss(pred: np.ndarray, labels: np.ndarray) -> tuple[float, np.ndarray]:
    # y is the one-hot enconding of labels
    actual = np.zeros((labels.shape[0], 10))
    actual[np.arange(labels.shape[0]), labels] = 1
    # cross-entropy loss
    loss = -np.mean(np.sum(actual * np.log(pred), axis=1))
    error = pred - actual
    return loss, error

In [14]:
def backward(error: np.ndarray, xs: tuple[np.ndarray]) -> tuple[np.ndarray]:
    x3, x2, x1, x0 = xs
    dx3 = error  # batch_size * 10, derivative of squared_error
    dw2 = x2.T @ dx3  # 128 * 10, derivative of dot
    dx2 = dx3 @ w2.T  # batch_size * 128, derivative of dot
    dx1 = (x1 > 0).astype(np.float64) * dx2 # 128 * batch_size
    dw1 = x0.T @ dx1  # 784 * 128
    assert dw2.shape == w2.shape
    assert dw1.shape == w1.shape
    return dw2, dw1

In [15]:
def update_weights(dws: tuple[np.ndarray], lr: float = 1e-3) -> tuple[np.ndarray]:
    global w1, w2
    ws = (w2, w1)
    # SGD
    for wi, dwi in zip(ws, dws):
        wi -= lr * dwi
    return ws

In [16]:
for i in (t := trange(iterations)):
    sample = np.random.randint(0, len(X_train), size=batch_size)
    xs = saved_forward(X_train[sample])
    loss, error = cross_entropy_loss(xs[0], Y_train[sample].astype(int))
    dws = backward(error, xs)
    update_weights(dws, lr=learning_rate)
    t.set_description(f"Iteration {i} loss {loss.item():.2f}")

Iteration 999 loss 0.17: 100%|██████████| 1000/1000 [00:05<00:00, 175.51it/s]


In [17]:
pred = forward(X_test).argmax(axis=1)
accuracy = (pred == Y_test).mean()
accuracy

0.9514